# CLARIFICATION QUESTIONS

- For the max speed we have found on the NYC website the types of vehilcles they use and checked max speed. where do we cite? in the code or
- Where to mention things liek to clean the dataset we kept trip amput 0 becUASE WE ARENT SURE WHGAT THE TRIPS WHERE someone doesnt pay at all count as 0 or a negativa value etc
- For the fare amout we think it cant be zero but, the way we removed iot is b exluding negarives and the Q1 lowest extreem quartile? surely thats optimal

# **Data Preprocessing**


In [ ]:
#import the datatset
import pandas as pd
import numpy as np

# Load raw monthly TLC Yellow Taxi dataset
df = pd.read_parquet("yellow_tripdata_2025-09.parquet")

# Visualise the dataset
print(df.shape)
df.head()


(4251015, 20)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,2,2025-09-01 00:19:20,2025-09-01 00:45:17,1.0,9.92,1.0,N,138,114,1,42.9,6.0,0.5,10.73,0.0,1.0,66.13,2.5,1.75,0.75
1,2,2025-09-01 00:15:20,2025-09-01 00:26:08,2.0,6.82,1.0,N,93,157,1,26.8,1.0,0.5,5.86,0.0,1.0,35.16,0.0,0.00,0.00
2,2,2025-09-01 00:06:07,2025-09-01 00:22:23,1.0,3.95,1.0,N,68,13,1,19.8,1.0,0.5,5.11,0.0,1.0,30.66,2.5,0.00,0.75
3,2,2025-09-01 00:49:47,2025-09-01 01:04:49,1.0,3.14,1.0,N,234,87,1,17.7,1.0,0.5,3.52,0.0,1.0,26.97,2.5,0.00,0.75
4,2,2025-09-01 00:05:00,2025-09-01 00:15:32,6.0,2.81,1.0,N,230,151,1,14.9,1.0,0.5,4.13,0.0,1.0,24.78,2.5,0.00,0.75


In [ ]:
print(df['VendorID'].value_counts(dropna=False))

VendorID
2    3374033
1     817547
7      56583
6       2852
Name: count, dtype: int64


In [ ]:
unexpected = df[~df['VendorID'].isin([1,2,6,7])]
print("Number of unexpected VendorIDs:")
print(unexpected['VendorID'].value_counts())

Number of unexpected VendorIDs:
Series([], Name: count, dtype: int64)


In [ ]:
missing_pct_per_vendor = (
    df.groupby('VendorID')
      .apply(lambda x: x.isna().mean() * 100)
)

print("\nPercentage of missing values per VendorID:")
print(missing_pct_per_vendor)


Percentage of missing values per VendorID:
          VendorID  tpep_pickup_datetime  tpep_dropoff_datetime  \
VendorID                                                          
1              0.0                   0.0                    0.0   
2              0.0                   0.0                    0.0   
6              0.0                   0.0                    0.0   
7              0.0                   0.0                    0.0   

          passenger_count  trip_distance  RatecodeID  store_and_fwd_flag  \
VendorID                                                                   
1               16.668644            0.0   16.668644           16.668644   
2               27.506222            0.0   27.506222           27.506222   
6              100.000000            0.0  100.000000          100.000000   
7                0.000000            0.0    0.000000            0.000000   

          PULocationID  DOLocationID  payment_type  fare_amount  extra  \
VendorID              

/tmp/ipython-input-3156252074.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.isna().mean() * 100)


In [ ]:
# Load taxi zone lookup
zones = pd.read_csv("taxi_zone_lookup.csv")

# Merge pickup spatial context
zones_PU = zones.rename(columns={
    'LocationID': 'PULocationID',
    'Borough': 'PU_Borough',
    'Zone': 'PU_Zone',
    'service_zone': 'PU_service_zone'
})[['PULocationID', 'PU_Borough', 'PU_Zone', 'PU_service_zone']]

# Merge dropoff spatial context
zones_DO = zones.rename(columns={
    'LocationID': 'DOLocationID',
    'Borough': 'DO_Borough',
    'Zone': 'DO_Zone',
    'service_zone': 'DO_service_zone'
})[['DOLocationID', 'DO_Borough', 'DO_Zone', 'DO_service_zone']]

# add to the main datframe
df = df.merge(zones_PU, on='PULocationID', how='left')
df = df.merge(zones_DO, on='DOLocationID', how='left')


# Visualise the dataset
print(df.shape)
df.head()


(4251015, 26)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone
0,2,2025-09-01 00:19:20,2025-09-01 00:45:17,1.0,9.92,1.0,N,138,114,1,...,66.13,2.5,1.75,0.75,Queens,LaGuardia Airport,Airports,Manhattan,Greenwich Village South,Yellow Zone
1,2,2025-09-01 00:15:20,2025-09-01 00:26:08,2.0,6.82,1.0,N,93,157,1,...,35.16,0.0,0.00,0.00,Queens,Flushing Meadows-Corona Park,Boro Zone,Queens,Maspeth,Boro Zone
2,2,2025-09-01 00:06:07,2025-09-01 00:22:23,1.0,3.95,1.0,N,68,13,1,...,30.66,2.5,0.00,0.75,Manhattan,East Chelsea,Yellow Zone,Manhattan,Battery Park City,Yellow Zone
3,2,2025-09-01 00:49:47,2025-09-01 01:04:49,1.0,3.14,1.0,N,234,87,1,...,26.97,2.5,0.00,0.75,Manhattan,Union Sq,Yellow Zone,Manhattan,Financial District North,Yellow Zone
4,2,2025-09-01 00:05:00,2025-09-01 00:15:32,6.0,2.81,1.0,N,230,151,1,...,24.78,2.5,0.00,0.75,Manhattan,Times Sq/Theatre District,Yellow Zone,Manhattan,Manhattan Valley,Yellow Zone


# Checking spatial logic

In [ ]:
# Count and drop trips in unknown zone (264) or outside of NYC (265)
unknown_ids = [264, 265]

# Count PU occurrences
pu_unknown_count = df[df['PULocationID'].isin(unknown_ids)].shape[0]

# Count DO occurrences
do_unknown_count = df[df['DOLocationID'].isin(unknown_ids)].shape[0]

# Count how many UNIQUE trips include either
either_unknown_count = df[
    df['PULocationID'].isin(unknown_ids) | df['DOLocationID'].isin(unknown_ids)
].shape[0]

print("Trips with PU in {264,265}:", pu_unknown_count)
print("Trips with DO in {264,265}:", do_unknown_count)
print("Trips touching zone 264 or 265:", either_unknown_count)

# Calculating the %
percent_unknown = (
    df[
        df['PULocationID'].isin(unknown_ids) |
        df['DOLocationID'].isin(unknown_ids)
    ].shape[0] / len(df)
) * 100

print("Percentage of trips touching zone 264 or 265:", percent_unknown)


Trips with PU in {264,265}: 8215
Trips with DO in {264,265}: 28247
Trips touching zone 264 or 265: 32074
Percentage of trips touching zone 264 or 265: 0.7545021600723593


In [ ]:
# <1% therfore drop trips involving zone 264 or 265
df = df[~df['PULocationID'].isin(unknown_ids)]
df = df[~df['DOLocationID'].isin(unknown_ids)]


In [ ]:
# Summary of missing values per column
missing_summary = df.isnull().sum().sort_values(ascending=False)

# Convert to percentages as well
missing_percent = (df.isnull().mean() * 100).sort_values(ascending=False)

# Combine into one table
missing_table = pd.DataFrame({
    'missing_count': missing_summary,
    'missing_percent': missing_percent})

missing_table


,missing_count,missing_percent
RatecodeID,1063667,25.211706
passenger_count,1063667,25.211706
store_and_fwd_flag,1063667,25.211706
congestion_surcharge,1063667,25.211706
Airport_fee,1063667,25.211706
VendorID,0,0.000000
trip_distance,0,0.000000
PULocationID,0,0.000000
tpep_pickup_datetime,0,0.000000
tpep_dropoff_datetime,0,0.000000


In [ ]:
# Check borough validity
pd.DataFrame({
    "Pickup": df["PU_Borough"].value_counts(),
    "Dropoff": df["DO_Borough"].value_counts()})

,Pickup,Dropoff
Bronx,30131,52736
Brooklyn,163140,259763
EWR,530,7892
Manhattan,3622170,3642988
Queens,402540,254254
Staten Island,430,1308


# Checking temporal logic

In [ ]:
# Convert timestamps
df['tpep_pickup_datetime']  = pd.to_datetime(df['tpep_pickup_datetime'], errors= 'coerce')
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'], errors= 'coerce')

# Remove rows where parsing failed
df = df.dropna(subset=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])


In [ ]:
# Remove rows where timestamp columns are empty / invalid
df = df[
    df['tpep_pickup_datetime'].notna() &
    df['tpep_dropoff_datetime'].notna()]

In [ ]:
# Check if there are any rows that contain data from other months other than September
df['tpep_pickup_datetime'].dt.month.value_counts()

In [ ]:
# Check if there are any rows that contain data from other yeears other than 2025
df['tpep_pickup_datetime'].dt.year.value_counts()

In [ ]:
# Drop rows containing data from anytime other than September and other than data from 2025
df = df[
    (df['tpep_pickup_datetime'].dt.year == 2025) &
    (df['tpep_pickup_datetime'].dt.month == 9)]

In [ ]:
# Check whether there are any where pick up is after drop off
invalid_timestamps = df[df['tpep_dropoff_datetime'] < df['tpep_pickup_datetime']]
invalid_count = invalid_timestamps.shape[0]

print("Number of rows with dropoff BEFORE pickup:", invalid_count)


In [ ]:
# Remove impossible timestamps (pickup after dropoff)
df = df[df['tpep_dropoff_datetime'] >= df['tpep_pickup_datetime']]

In [ ]:
# Compute trip duration
df['duration_min'] = (
    df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
).dt.total_seconds() / 60

df = df[df['duration_min'] > 0]  # drop impossible trips

df.head(5).round({'duration_min': 2})



In [ ]:
df['duration_min'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2)

In [ ]:
#Duration
print("\n=== Trip Duration (minutes) Summary by VendorID ===")
print(df.groupby("VendorID")['duration_min'].describe())


In [ ]:
# Identify trips loger than 4 hours long
invalid_long = df[df['duration_min'] > 240]
print("Trips with duration > 240 min:", invalid_long.shape[0])
print("Percentage:", invalid_long.shape[0] / len(df) * 100)

In [ ]:
# Drop trips loger than 4 hours long
df = df[(df['duration_min'] > 0) & (df['duration_min'] <= 240)]

# Numeric Integrity Checks & Outlier Removal

In [ ]:
# Ensure numeric columns contain only numeric values
temp = df.copy()
numeric_cols = [
    'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
    'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
    'congestion_surcharge', 'airport_fee', 'cbd_congestion_fee',
    'RatecodeID', 'payment_type', 'PULocationID', 'DOLocationID']

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')


In [ ]:
# Compute speed
df['speed_mph'] = df['trip_distance'] / (df['duration_min'] / 60)

In [ ]:
# Count invalid rows where the speed is less than 0 and greater than 120
invalid_speed = df[(df['speed_mph'] <= 0) | (df['speed_mph'] > 120)]
print("Invalid speed rows:", invalid_speed.shape[0])
print("Percentage:", invalid_speed.shape[0] / len(df) * 100)

In [ ]:
 # Drop invalid speed
df = df[(df['speed_mph'] > 0) & (df['speed_mph'] <= 120)]

In [ ]:
df['speed_mph'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2)

In [ ]:
Q1 = df['speed_mph'].quantile(0.01)
Q99 = df['speed_mph'].quantile(0.99)
df = df[(df['speed_mph'] >= Q1) & (df['speed_mph'] <= Q99)]

In [ ]:
# Count invalid rows where the distance is less than 0 and greater than 120
invalid_distance = df[(df['trip_distance'] <= 0) | (df['trip_distance'] > 120)]
print("Invalid distance rows:", invalid_distance.shape[0])
print("Percentage:", invalid_distance.shape[0] / len(df) * 100)

In [ ]:
 # Drop invalid distance
df = df[(df['trip_distance'] > 0) & (df['trip_distance'] <= 120)]


In [ ]:
df['trip_distance'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2)

In [ ]:
# Drop extremes
Q1 = df['trip_distance'].quantile(0.01)
Q99 = df['trip_distance'].quantile(0.99)
df = df[(df['trip_distance'] >= Q1) & (df['trip_distance'] <= Q99)]

In [ ]:
# Fare logic test
# Check how many negative values are present in the fare_amount column and total_amount column
negative_fare_count = (df['fare_amount'] < 0).sum()
print ("Number of negative fare amounts:", negative_fare_count)
negative_total_fare_count = (df['total_amount'] < 0).sum()
print ("Number of negative total fare amounts:", negative_total_fare_count)

In [ ]:
#Check if All negative total fare values occur when base fare amount is negative
# Condition: For all rows where base fare < 0, total fare must also be < 0
condition = (df.loc[df['total_amount'] < 0, 'fare_amount'] < 0).all()

if condition:
    print("Yes: Every negative total fare has a negative base fare.")
else:
    print("No: There are cases where total fare is negative but base fare is not.")

In [ ]:
#Check if All negative base fare values occur when total fare amount is negative
# Condition: For all rows where total fare < 0, base fare must also be < 0
condition = (df.loc[df['fare_amount'] < 0, 'total_amount'] < 0).all()

if condition:
    print("Yes: Every negative base fare has a negative total fare.")
else:
    print("No: There are cases where base fare is negative but total fare is not.")

In [ ]:
# Drop invalid fares
df = df[(df['total_amount'] > 0) & (df['fare_amount'] >= 0)]

In [ ]:
# Visualise the dataset
print(df.shape)

#Check the distribution of fare amount and total amount
pd.DataFrame({
    "fare_amount": df['fare_amount'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2),
    "total_amount": df['total_amount'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2)})

In [ ]:
# Drop extremes in fare_amount
Q1 = df['fare_amount'].quantile(0.01)
Q99 = df['fare_amount'].quantile(0.99)
df = df[(df['fare_amount'] >= Q1) & (df['fare_amount'] <= Q99)]

In [ ]:
# # Drop extremes in total_amount
Q1 = df['total_amount'].quantile(0.01)
Q99 = df['total_amount'].quantile(0.99)
df = df[(df['total_amount'] >= Q1) & (df['total_amount'] <= Q99)]

In [ ]:
# Visualise the dataset
print(df.shape)

In [ ]:
pd.DataFrame({
    "fare_amount": df['fare_amount'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2),
    "total_amount": df['total_amount'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2)})

In [ ]:
# Remove all rows with either negative extra or cbd_congestion_fee
df = df[(df['extra'] >= 0) & (df['cbd_congestion_fee'] >= 0)]

# Visualise the dataset
print(df.shape)

In [ ]:
df['cbd_congestion_fee'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2)

In [ ]:
df['extra'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2)

In [ ]:
# Address  extreme values in extra and cbd_congestion_fee
# In extra
Q99 = df['extra'].quantile(0.99)
df = df[(df['extra'] <= Q99)]

# In cbd_congestion_fee
Q99 = df['cbd_congestion_fee'].quantile(0.99)
df = df[(df['cbd_congestion_fee'] <= Q99)]

In [ ]:
pd.DataFrame({
    "extra": df['extra'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2),
    "cbd_congestion_fee": df['cbd_congestion_fee'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2)})

In [ ]:
# Count invalid rows where the number of passangers is 0 and greater than 9, considering 5 adults and 4 children under 7 on laps of the adults (New York City Taxi and Limousine Commission 2025)
invalid_speed = df[(df['speed_mph'] <= 0) | (df['speed_mph'] > 120)]
print("Invalid speed rows:", invalid_speed.shape[0])
print("Percentage:", invalid_speed.shape[0] / len(df) * 100)

In [ ]:
#check if any row with numerical value has a zero value
zero_values = df.select_dtypes(include=['number']).eq(0).any()
print(zero_values)

In [ ]:
df['passenger_count'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2)


In [ ]:
#remove all rows with zero passenger count
df = df[df['passenger_count'] > 0]

#address abnormal extreme values in passenger count; keep max passenger count at 5 as describe in The Official Website of the City of New York
upper_limit = 9
df = df[df['passenger_count'] <= upper_limit]

df['passenger_count'].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).round(2)

# Buliding Comparison graphs

In [ ]:
# Creating the 'before' dataframe with minimal data preprocessing
import pandas as pd

# BEFORE PREPROCESSING DATA
# Load raw data
df_before = pd.read_parquet("yellow_tripdata_2025-09.parquet")

# Parse timestamps
df_before['tpep_pickup_datetime']  = pd.to_datetime(
    df_before['tpep_pickup_datetime'], errors='coerce')
df_before['tpep_dropoff_datetime'] = pd.to_datetime(
    df_before['tpep_dropoff_datetime'], errors='coerce')

# Drop rows where timestamps are invalid
df_before = df_before.dropna(subset=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

# Coerce key numeric columns
numeric_cols = ['trip_distance', 'fare_amount', 'total_amount',
    'PULocationID', 'DOLocationID']

for col in numeric_cols:
    if col in df_before.columns:
        df_before[col] = pd.to_numeric(df_before[col], errors='coerce')

# Drop rows with missing critical numeric values
df_before = df_before.dropna(subset=['trip_distance', 'PULocationID', 'DOLocationID'])

# Compute duration
df_before['duration_min'] = (
    df_before['tpep_dropoff_datetime'] - df_before['tpep_pickup_datetime']).dt.total_seconds() / 60

# Compute speed (only where duration_min > 0 to avoid division by zero)
df_before = df_before[df_before['duration_min'] > 0].copy()
df_before['speed_mph'] = df_before['trip_distance'] / (df_before['duration_min'] / 60)

# Merge spatial info (zones, boroughs, service_zone)
zones = pd.read_csv("taxi_zone_lookup.csv")

zones_PU = zones.rename(columns={
    'LocationID': 'PULocationID',
    'Borough': 'PU_Borough',
    'Zone': 'PU_Zone',
    'service_zone': 'PU_service_zone'
})[['PULocationID', 'PU_Borough', 'PU_Zone', 'PU_service_zone']]

zones_DO = zones.rename(columns={
    'LocationID': 'DOLocationID',
    'Borough': 'DO_Borough',
    'Zone': 'DO_Zone',
    'service_zone': 'DO_service_zone'
})[['DOLocationID', 'DO_Borough', 'DO_Zone', 'DO_service_zone']]

df_before = df_before.merge(zones_PU, on='PULocationID', how='left')
df_before = df_before.merge(zones_DO, on='DOLocationID', how='left')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# sensible range for plotting
max_dist = 50  # or e.g. df_before['trip_distance'].quantile(0.99)

before_mask = (df_before['trip_distance'] > 0) & (df_before['trip_distance'] <= max_dist)
after_mask  = (df['trip_distance'] > 0) & (df['trip_distance'] <= max_dist)

# BEFORE CLEANING (visual only – still using raw df_before, just clipped)
df_before.loc[before_mask, 'trip_distance'].hist(
    bins=100,
    ax=ax[0],
    color='orange'
)
ax[0].set_title("Trip Distance BEFORE Cleaning (0–50 miles)")
ax[0].set_xlabel("Trip Distance (miles)")
ax[0].set_ylabel("Frequency")
ax[0].set_xlim(0, max_dist)

# AFTER CLEANING
df.loc[after_mask, 'trip_distance'].hist(
    bins=100,
    ax=ax[1],
    color='green'
)
ax[1].set_title("Trip Distance AFTER Cleaning (0–50 miles)")
ax[1].set_xlabel("Trip Distance (miles)")
ax[1].set_ylabel("Frequency")
ax[1].set_xlim(0, max_dist)
ax[0].set_yscale('log')
ax[1].set_yscale('log')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

df['trip_distance'].hist(
    bins=200,
    figsize=(10,5))

plt.title("Trip Distance Distribution After Outlier Removal")
plt.xlabel("Trip Distance (miles)")
plt.ylabel("Frequency")
plt.xlim(0, 20)   # Zoomed-in view
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Set a range
max_dur = 1000

before_mask = (df_before['duration_min'] > 0) & (df_before['duration_min'] <= max_dur)
after_mask  = (df['duration_min'] > 0) & (df['duration_min'] <= max_dur)

# BEFORE CLEANING
df_before.loc[before_mask, 'duration_min'].hist(
    bins=100,
    ax=ax[0],
    color='orange')
ax[0].set_title("Trip Duration BEFORE Cleaning (0–120 minutes)")
ax[0].set_xlabel("Duration (minutes)")
ax[0].set_ylabel("Frequency")
ax[0].set_xlim(0, max_dur)
ax[0].set_yscale('log')

# AFTER CLEANING
df.loc[after_mask, 'duration_min'].hist(
    bins=100,
    ax=ax[1],
    color='green')

ax[1].set_title("Trip Duration AFTER Cleaning (0–1000 minutes)")
ax[1].set_xlabel("Duration (minutes)")
ax[1].set_ylabel("Frequency")
ax[1].set_xlim(0, max_dur)
ax[1].set_yscale('log')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# sensible range (visual only)
max_speed = 100  # almost no taxi exceeds this if data is clean

before_mask = (df_before['speed_mph'] > 0) & (df_before['speed_mph'] <= max_speed)
after_mask  = (df['speed_mph'] > 0) & (df['speed_mph'] <= max_speed)

# BEFORE CLEANING
df_before.loc[before_mask, 'speed_mph'].hist(
    bins=100,
    ax=ax[0],
    color='orange'
)
ax[0].set_title("Speed BEFORE Cleaning (0–100 mph)")
ax[0].set_xlabel("Speed (mph)")
ax[0].set_ylabel("Frequency")
ax[0].set_xlim(0, max_speed)
ax[0].set_yscale('log')

# AFTER CLEANING
df.loc[after_mask, 'speed_mph'].hist(
    bins=100,
    ax=ax[1],
    color='green'
)
ax[1].set_title("Speed AFTER Cleaning (0–100 mph)")
ax[1].set_xlabel("Speed (mph)")
ax[1].set_ylabel("Frequency")
ax[1].set_xlim(0, max_speed)
ax[1].set_yscale('log')

plt.tight_layout()
plt.show()

In [ ]:
# Summary of missing values per column
missing_summary = df.isnull().sum().sort_values(ascending=False)

# Convert to percentages as well
missing_percent = (df.isnull().mean() * 100).sort_values(ascending=False)

# Combine into one table
missing_table = pd.DataFrame({
    'missing_count': missing_summary,
    'missing_percent': missing_percent})

missing_table

# Vendor Check

In [ ]:
vendor_6_df = df[df['VendorID'] == 6]
if not vendor_6_df.empty:
    print(vendor_6_df.sample(min(10, len(vendor_6_df)), random_state=42))
else:
    print("No rows found for VendorID == 6 after cleaning.")
    # As a fallback, also show the value counts of VendorID in the current DataFrame
    print("\nCurrent VendorID distribution:")
    print(df['VendorID'].value_counts())

In [ ]:
df['VendorID'].value_counts()


1.1.2 Missing Values per Vendor

In [ ]:
missing_by_vendor = df.groupby("VendorID").apply(lambda x: x.isnull().mean())
print("\n=== Missingness per VendorID (proportion of nulls) ===")
print(missing_by_vendor)

1.1.3. Distribution diagnostics per Vendor

In [ ]:
#Speed
print("\n=== Trip Speed (mph) Summary by VendorID ===")
print(df.groupby("VendorID")['speed_mph'].describe())


In [ ]:
import matplotlib.pyplot as plt

for v in [1, 2]:
    df[df['VendorID'] == v]['trip_distance'].hist(
        bins=200,
        alpha=0.5,
        label=f"Vendor {v}",
        figsize=(10,5)
    )

plt.legend()
plt.title("Trip Distance Distribution by Vendor")
plt.xlabel("Trip Distance (miles)")
plt.ylabel("Frequency")
plt.xlim(0, 30)   # focus on realistic range
plt.show()

Normalising

In [ ]:

df['VendorID'].value_counts(normalize=True)

missing_by_vendor = df.groupby("VendorID").apply(lambda x: x.isnull().mean()).T
missing_by_vendor




In [ ]:
df['pickup_bin'] = df['tpep_pickup_datetime'].dt.floor('30T')

# Group by zone and time bin, count trips
trips_per_bin = (
    df.groupby(['PULocationID', 'pickup_bin'])
      .size()
      .reset_index(name='trips_count')
)

# Show the first few rows of the result
print(trips_per_bin.head(10))

In [ ]:
import matplotlib.pyplot as plt

zone_id = 132  # example zone
zone_data = trips_per_bin[trips_per_bin['PULocationID'] == zone_id]

plt.figure(figsize=(12,6))
plt.plot(zone_data['pickup_bin'], zone_data['trips_count'])
plt.title(f"Taxi demand per 30 min in Zone {zone_id}")
plt.xlabel("Time")
plt.ylabel("Trips count")
plt.xticks(rotation=45)
plt.show()

### Aggregate Data by Half Hourly Intervals

In [ ]:
# Make sure pickup datetime is datetime type
temp['tpep_pickup_datetime'] = pd.to_datetime(temp['tpep_pickup_datetime'])

# Half-hour bins
temp['pickup_bin'] = temp['tpep_pickup_datetime'].dt.floor('30T')

# Aggregate by pickup location and half-hour
trips_per_bin = (
    temp.groupby(['PULocationID', 'pickup_bin'])
      .agg(
          # volume
          trips_count=('VendorID', 'size'),

          # passengers
          avg_passenger_count=('passenger_count', 'mean'),
          total_passengers=('passenger_count', 'sum'),

          # distance
          avg_trip_distance=('trip_distance', 'mean'),
          total_trip_distance=('trip_distance', 'sum'),

          # time
          avg_duration_min=('duration_min', 'mean'),
          total_duration_min=('duration_min', 'sum'),

          # fares & money columns
          avg_fare_amount=('fare_amount', 'mean'),
          total_fare_amount=('fare_amount', 'sum'),
          total_extra=('extra', 'sum'),
          total_mta_tax=('mta_tax', 'sum'),
          avg_tip_amount=('tip_amount', 'mean'),
          total_tip_amount=('tip_amount', 'sum'),
          total_tolls_amount=('tolls_amount', 'sum'),
          total_improvement_surcharge=('improvement_surcharge', 'sum'),
          total_total_amount=('total_amount', 'sum'),
          total_congestion_surcharge=('congestion_surcharge', 'sum'),
          total_airport_fee=('airport_fee', 'sum'),
          total_cbd_congestion_fee=('cbd_congestion_fee', 'sum'),
      )
      .reset_index()
)
# assume temp is already filtered to a single month
month_start = temp['tpep_pickup_datetime'].min().to_period('M').to_timestamp()       # first day, 00:00
month_end   = (month_start + pd.offsets.MonthEnd(0)) + pd.Timedelta(hours=23, minutes=30)

# all half-hour bins for that month
all_bins = pd.date_range(start=month_start,
                         end=month_end,
                         freq='30T')

# all pickup locations present in this month
all_pu = temp['PULocationID'].unique()

# full grid: every PU × every half-hour in the month
full_index = pd.MultiIndex.from_product(
    [all_pu, all_bins],
    names=['PULocationID', 'pickup_bin']
)

# reindex aggregated data onto full grid
trips_per_bin_full = (
    trips_per_bin
    .set_index(['PULocationID', 'pickup_bin'])
    .reindex(full_index)
    .reset_index()
)

# fill numeric NaNs with 0 (no rides in that interval)
numeric_cols = [
    'trips_count',
    'avg_passenger_count',
    'total_passengers',
    'avg_trip_distance',
    'total_trip_distance',
    'avg_duration_min',
    'total_duration_min',
    'avg_fare_amount',
    'total_fare_amount',
    'total_extra',
    'total_mta_tax',
    'avg_tip_amount',
    'total_tip_amount',
    'total_tolls_amount',
    'total_improvement_surcharge',
    'total_total_amount',
    'total_congestion_surcharge',
    'total_airport_fee',
    'total_cbd_congestion_fee',
]

trips_per_bin_full[numeric_cols] = trips_per_bin_full[numeric_cols].fillna(0)
display(trips_per_bin.head(10))


In [ ]:
display(trips_per_bin_full.columns)

#*NOOK*: Adds Events

In [ ]:
big_events = {
    "NYC_Broadway_Week": ("2025-09-08", "2025-09-21"),
    "NYC_Fashion_Week": ("2025-09-11", "2025-10-16"),
    "Brooklyn_Book_Festival": ("2025-09-14", "2025-09-22"),
    "NYC_Ballet": ("2025-09-16", "2025-10-12"),
    "Atlantic_Festival": ("2025-09-18", "2025-09-20"),
    "Climate_Week": ("2025-09-21", "2025-09-28"),
    "NYC_Film_Festival": ("2025-09-26", "2025-10-13"),
    "Global_Citizen_Festival": ("2025-09-27", "2025-09-27")
}
# Convert timestamp if needed
trips_per_bin_full['pickup_bin'] = pd.to_datetime(trips_per_bin_full['pickup_bin'])

for event_name, (start, end) in big_events.items():
    start = pd.to_datetime(start)
    end   = pd.to_datetime(end)

    # Boolean flag for whether pickup_bin is within event period
    trips_per_bin_full[event_name] = (
        (trips_per_bin_full['pickup_bin'] >= start) &
        (trips_per_bin_full['pickup_bin'] <= end)
    )

In [ ]:
display(trips_per_bin_full)

In [ ]:
import holidays

us_holidays = holidays.US()   # contains all US federal holidays
# Ensure datetime

# Day-of-week name
trips_per_bin_full['pickup_day_name'] = trips_per_bin_full['pickup_bin'].dt.day_name()

# Boolean weekday flags
trips_per_bin_full['is_monday']    = trips_per_bin_full['pickup_bin'].dt.weekday == 0
trips_per_bin_full['is_tuesday']   = trips_per_bin_full['pickup_bin'].dt.weekday == 1
trips_per_bin_full['is_wednesday'] = trips_per_bin_full['pickup_bin'].dt.weekday == 2
trips_per_bin_full['is_thursday']  = trips_per_bin_full['pickup_bin'].dt.weekday == 3
trips_per_bin_full['is_friday']    = trips_per_bin_full['pickup_bin'].dt.weekday == 4
trips_per_bin_full['is_saturday']  = trips_per_bin_full['pickup_bin'].dt.weekday == 5
trips_per_bin_full['is_sunday']    = trips_per_bin_full['pickup_bin'].dt.weekday == 6

# Optional: weekend flag
trips_per_bin_full['is_weekend'] = trips_per_bin_full['pickup_bin'].dt.weekday >= 5

# Day of the month
trips_per_bin_full['day_of_month'] = trips_per_bin_full['pickup_bin'].dt.day

# Month (1–12)
trips_per_bin_full['month'] = trips_per_bin_full['pickup_bin'].dt.month

# Month name
trips_per_bin_full['month_name'] = trips_per_bin_full['pickup_bin'].dt.month_name()

# Week of year
trips_per_bin_full['week_of_year'] = trips_per_bin_full['pickup_bin'].dt.isocalendar().week

# US Holiday flag
trips_per_bin_full['is_holiday'] = trips_per_bin_full['pickup_bin'].dt.date.isin(us_holidays)
trips_per_bin_full['is_NOT_holiday'] = ~trips_per_bin_full['pickup_bin'].dt.date.isin(us_holidays)

# Extract weekday (Mon=0 ... Sun=6) and hour (0-23)
trips_per_bin_full['weekday'] = trips_per_bin_full['pickup_bin'].dt.weekday
trips_per_bin_full['hour']    = trips_per_bin_full['pickup_bin'].dt.hour

weekday = trips_per_bin_full['weekday']
hour    = trips_per_bin_full['hour']

# Rush hour: Mon–Fri, 4pm–8pm (16:00–19:59)
trips_per_bin_full['is_rush_hour'] = (
    hour.between(16, 19) &
    weekday.between(0, 4)
)

# Overnight: 8pm–6am (20:00–05:59)
trips_per_bin_full['is_overnight'] = (hour >= 20) | (hour < 6)

# Friday evening: Fri 18:00 → Sat 02:59
trips_per_bin_full['is_friday_evening'] = (
    ((weekday == 4) & (hour >= 18)) |
    ((weekday == 5) & (hour < 3))
)

# Saturday evening: Sat 18:00 → Sun 02:59
trips_per_bin_full['is_saturday_evening'] = (
    ((weekday == 5) & (hour >= 18)) |
    ((weekday == 6) & (hour < 3))
)

In [ ]:
# Creating new variable Half Hour of Day
trips_per_bin_full['HalfHourOfDay'] = trips_per_bin_full['pickup_bin'].dt.hour * 2 + (trips_per_bin_full['pickup_bin'].dt.minute // 30)

# cyclical encoding: To make sure the model recognize taht 23:30 (47) is next to 00:00 (0)
trips_per_bin_full['sin_halfhour'] = np.sin(2 * np.pi * trips_per_bin_full['HalfHourOfDay'] / 48)
trips_per_bin_full['cos_halfhour'] = np.cos(2 * np.pi * trips_per_bin_full['HalfHourOfDay'] / 48)
bool_cols = trips_per_bin_full.select_dtypes(include='bool').columns
trips_per_bin_full[bool_cols] = trips_per_bin_full[bool_cols].astype(int)
display(trips_per_bin_full)


In [ ]:
display(trips_per_bin_full.columns)

#Tiana: Adds Rain

In [ ]:
# Define coordinate lookup for each borough / special location
coord_map = {
    "EWR":             (40.6895, -74.1745),
    "Manhattan":       (40.7831, -73.9712),
    "Brooklyn":        (40.6782, -73.9442),
    "Queens":          (40.7282, -73.7949),
    "Bronx":           (40.8448, -73.8648),
    "Staten Island":   (40.5795, -74.1502),
    "Unknown":         (None, None),
    None:              (None, None),      # for NaN Borough rows
}

# Extract lat/lon into new columns
zones['lat'] = zones['Borough'].apply(lambda x: coord_map.get(x, (None, None))[0])
zones['lon'] = zones['Borough'].apply(lambda x: coord_map.get(x, (None, None))[1])

In [ ]:
from datetime import datetime
from meteostat import Hourly, Point
import pandas as pd
# pip install meteostat
trips = trips_per_bin_full.merge(
    zones[['LocationID', 'lat', 'lon']],
    left_on='PULocationID',
    right_on='LocationID',
    how='left'
)
# Make sure pickup_bin is datetime
trips_per_bin_full['pickup_bin'] = pd.to_datetime(trips_per_bin_full['pickup_bin'])

# Time range from your data
start = trips_per_bin_full['pickup_bin'].min().to_pydatetime()
end   = trips_per_bin_full['pickup_bin'].max().to_pydatetime()

weather_frames = []

# Use unique zones that actually have coordinates
zones_with_coords = zones.dropna(subset=['lat', 'lon'])

for loc_id, row in zones_with_coords.set_index('LocationID').iterrows():
    lat = row['lat']
    lon = row['lon']

    point = Point(lat, lon)

    # Fetch hourly weather for this location
    w = Hourly(point, start, end).fetch()

    # Meteostat returns a DatetimeIndex called 'time'
    w = (
        w.reset_index()
         .rename(columns={'time': 'hour'})
    )
    w['LocationID'] = loc_id

    # Keep only what you need
    weather_frames.append(
        w[['LocationID', 'hour', 'temp', 'prcp']]
    )

# Combine all locations into one weather table
weather_all = pd.concat(weather_frames, ignore_index=True)

# Create matching 'hour' column on trips_per_bin_full (floor half-hour bins to hour)
trips_per_bin_full['hour'] = trips_per_bin_full['pickup_bin'].dt.floor('H')

# Merge weather into trips_per_bin_full
trips_per_bin_full = trips_per_bin_full.merge(
    weather_all,
    left_on=['PULocationID', 'hour'],
    right_on=['LocationID', 'hour'],
    how='left'
)

# Rename final weather columns
trips_per_bin_full = trips_per_bin_full.rename(columns={
    'temp': 'temp_C',
    'prcp': 'precip_mm'
})

# Optional: drop LocationID if you don't need the duplicate
trips_per_bin_full = trips_per_bin_full.drop(columns=['LocationID'])

display(trips_per_bin_full.head())


#Sunny Add in  Airport Data

In [ ]:
# filter for Airport Pickups
df_airports = df[df['PU_service_zone'] == 'Airports']
print (df_airports)

In [ ]:
df_airports['tpep_dropoff_datetime'] = pd.to_datetime(df_airports['tpep_dropoff_datetime'])
df_airports['dropoff_bin'] = df_airports['tpep_dropoff_datetime'].dt.floor('30T')
do_counts = (
    df_airports
      .groupby(['DOLocationID', 'dropoff_bin'])
      .size()
      .reset_index(name='airport_drop_off_count')
)
display(do_counts)

In [ ]:
trips_per_bin_full = trips_per_bin_full.merge(
    do_counts,
    left_on=['PULocationID', 'pickup_bin'],
    right_on=['DOLocationID', 'dropoff_bin'],
    how='left'
)

# clean up and fill
trips_per_bin_full = trips_per_bin_full.drop(columns=['DOLocationID', 'dropoff_bin'])
trips_per_bin_full['airport_drop_off_count'] = trips_per_bin_full['airport_drop_off_count'].fillna(0).astype(int)
display(trips_per_bin_full)

# NOOK Add 4 Interactive Features

In [ ]:
#Create interaction features to capture combined effects.
#These can reveal non-linear relationships between variables.

# HalfHourOfDay x is_weekend (Rush Patterns may differ on weekends)
trips_per_bin_full['HalfHourOfDay_isweekend '] = trips_per_bin_full['HalfHourOfDay'] * trips_per_bin_full['is_weekend']

# HalfHourOfDay x is_holiday  (Holiday traffic may behave differently)
trips_per_bin_full['HalfHourOfDay_isholiday '] = trips_per_bin_full['HalfHourOfDay'] * trips_per_bin_full['is_holiday']


# PULocationID x HalfHourOfDay (Demand in some  areas spike at night, others in the morning)
trips_per_bin_full['PULocationID_HalfHourOfDay '] = trips_per_bin_full['PULocationID'] * trips_per_bin_full['HalfHourOfDay']

# PULocationID x is_weekend  (Reflects difference due to weekend hotspot )
trips_per_bin_full['PULocationID_isweekend'] = trips_per_bin_full['PULocationID'] * trips_per_bin_full['is_weekend']

display(trips_per_bin_full.head())

In [ ]:
display(trips_per_bin_full.columns) #need to make sure airport demand is included in the table as one of the columns

# NOOK Add Rolling Average Demands and Lag Demands

In [ ]:
# Rolling Average Demands

rolling_windows = {
    '1.5h': 3,   # 3 bins = 1.5 hours
    '3h': 6,     # 6 bins = 3 hours
    '6h': 12,    # 12 bins = 6 hours
    '12h': 24,   # 24 bins = 12 hours
    '24h': 48    # 48 bins = 24 hours
}

for label, window in rolling_windows.items():
    trips_per_bin_full[f'rolling_avg_{label}_lag'] = (
        trips_per_bin
        .groupby('PULocationID')['trips_count']
        .transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
    )

# Lag Demands

lag_bins = {
    '0.5h': 1,
    '1h': 2,
    '1.5h': 3,
    '2h': 4,
    '3h': 6,
    '6h': 12
}

for label, bins in lag_bins.items():
    trips_per_bin_full[f'lag_trips_{label}'] = (
        trips_per_bin
        .groupby('PULocationID')['trips_count']
        .transform(lambda x: x.shift(bins))
    )

# Optional: Fill NaNs at start of each pickup zone #
trips_per_bin_full.fillna(0, inplace=True)


cols_to_check = ['PULocationID', 'pickup_bin', 'trips_count'] + \
                [f'rolling_avg_{w}_lag' for w in rolling_windows] + \
                [f'lag_trips_{l}' for l in lag_bins]

display(trips_per_bin_full[cols_to_check].head(15))

In [ ]:
display(trips_per_bin_full)

In [ ]:
print(df.shape)

In [ ]:
#Further Disscusion to determine which features to drop and keep

In [ ]:
from sklearn.preprocessing import StandardScaler

split_date = '2025-09-25'

# Split trips_per_bin_full instead of df
trips_train = trips_per_bin_full[trips_per_bin_full['pickup_bin'] < split_date]
trips_test  = trips_per_bin_full[trips_per_bin_full['pickup_bin'] >= split_date]

# List numeric features to standardize
numeric_cols = [
    'trips_count', 'avg_passenger_count', 'total_passengers', 'avg_trip_distance',
    'total_trip_distance', 'avg_duration_min', 'total_duration_min', 'avg_fare_amount',
    'total_fare_amount', 'total_extra', 'total_mta_tax', 'avg_tip_amount',
    'total_tip_amount', 'total_tolls_amount', 'total_improvement_surcharge',
    'total_total_amount', 'total_congestion_surcharge', 'total_airport_fee',
    'total_cbd_congestion_fee', 'airport_drop_off_count',
    'rolling_avg_1.5h_lag', 'rolling_avg_3h_lag', 'rolling_avg_6h_lag',
    'rolling_avg_12h_lag', 'rolling_avg_24h_lag',
    'lag_trips_0.5h','lag_trips_1h','lag_trips_1.5h','lag_trips_2h',
    'lag_trips_3h','lag_trips_6h'
]

scaler = StandardScaler()

# Fit scaler on training data
trips_train_scaled = trips_train.copy()
trips_train_scaled[numeric_cols] = scaler.fit_transform(trips_train_scaled[numeric_cols])

# Transform validation/test data
trips_val_scaled = trips_test.copy()
trips_val_scaled[numeric_cols] = scaler.transform(trips_val_scaled[numeric_cols])